In [1]:

import numpy as np 
import pandas as pd 
import seaborn as sns
from catboost import CatBoostClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    return summary

In [3]:
test1=pd.read_csv("test.csv")
test = test1.copy()
test.head()
df = pd.read_csv("train.csv")
df.head()

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,...,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,...,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,...,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,...,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,...,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived


In [5]:
test1=pd.read_csv("/kaggle/input/playground-series-s3e22/test.csv")
test = test1.copy()
test.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/playground-series-s3e22/test.csv'

In [ ]:
df = pd.read_csv("/kaggle/input/playground-series-s3e22/train.csv")
df.head()

In [4]:
resumetable(df)


Dataset Shape: (1235, 29)


,Name,dtypes,Missing,Uniques,First Value,Second Value
0,id,int64,0,1235,0,1
1,surgery,object,0,2,yes,yes
2,age,object,0,2,adult,adult
3,hospital_number,int64,0,255,530001,533836
4,rectal_temp,float64,0,43,38.1,37.5
5,pulse,float64,0,50,132.0,88.0
6,respiratory_rate,float64,0,37,24.0,12.0
7,temp_of_extremities,object,0,5,cool,cool
8,peripheral_pulse,object,0,5,reduced,normal
9,mucous_membrane,object,0,7,dark_cyanotic,pale_cyanotic


In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# Remove 'outcome' from the list
categorical_columns.remove('outcome')

# Now, categorical_columns contains all categorical column names except 'outcome'
categorical_columns

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
categorical_columnst = test.select_dtypes(include=['object']).columns.tolist()

# Remove 'outcome' from the list
categorical_columns.remove('outcome')

# Fill missing values with 'missing' in categorical columns using a loop
for col in categorical_columns:
    df[col].fillna('missing', inplace=True)
for col in categorical_columnst:
    test[col].fillna('missing', inplace=True)

In [ ]:
y=df["outcome"]
df.drop(["id","outcome"],axis=1,inplace=True)
test.drop(["id"],axis=1,inplace=True)

x=df
x

In [ ]:

# from sklearn.preprocessing import LabelEncoder

# # Assuming 'y' contains your target variable
# le = LabelEncoder()
# y = le.fit_transform(y)

# # Now, 'y_encoded' contains the label-encoded target variable
# y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33,random_state=42)


In [ ]:
from catboost import CatBoostClassifier

# Create a CatBoostClassifier with the desired evaluation metric
model_cb = CatBoostClassifier(iterations=100, 
                              learning_rate=0.1, 
                              eval_metric='Accuracy',  # Use a suitable metric for your problem
                              cat_features=categorical_columns)

# Fit the model with the training data and evaluation data
model_cb.fit(x_train, y_train, eval_set=(x_test, y_test), plot=True)


In [ ]:
y_pred = model_cb.predict(x_test)
accuracy_score(y_test, y_pred)

In [ ]:
resumetable(test)

In [ ]:
# Make predictions on the test set
predictions = model_cb.predict(test)

# Reshape predictions to be one-dimensional
predictions = predictions.reshape(-1, 1)

# Create a DataFrame for the submission
output = pd.DataFrame({'id': test1.id.values, 'Outcome': predictions.flatten()})

# Save the predictions to a CSV file
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")


In [ ]:
predictions.shape